# Limpiar Dataset de Productos

Este notebook limpia el archivo CSV `DetalleFacturas.csv` y genera `DetalleFacturas_clean.csv`.

**Nota:** Este notebook es opcional. Los datos ya están cargados en PostgreSQL mediante `bd-sistema.sql`.

In [ ]:
import pandas as pd
import re

## 1. Cargar datos originales

In [ ]:
df = pd.read_csv("../data/DetalleFacturas.csv", encoding="latin-1")
print(f"Total de productos cargados: {len(df)}")
df.head()

## 2. Definir reglas de limpieza

In [ ]:
# Patrones de descripciones inválidas (no son productos)
invalid = [
    # Tarifas y cargos
    'tarifa', 'cargo', 'contribución', 'gubernamental',
    
    # Servicios
    'hospedaje', 'lodging', 'consumo de alimentos', 'consumo del día',
    'alimentos y bebidas', 'servicios de cafetería', 'restaurante',
    
    # Códigos/Referencias
    'pnr:', 'folio', 'ticket', 'según número', 'seg�n',
    
    # Descriptores genéricos
    'consumo', 'no. mesa', 'cant pcio', 'total', 'subtotal',
    'descuento', 'receipt', 'tpv adyen',
    
    # Servicios específicos
    'corte en guillotina', 'vinil en coroplast',
]

def is_valid_product(description):
    """Valida si una descripción es un producto real"""
    if pd.isna(description):
        return False
    
    desc_lower = str(description).lower()
    desc_str = str(description)

    # Filtrar patrones inválidos
    for pattern in invalid:
        if pattern in desc_lower:
            return False
    
    # Filtrar códigos alfanuméricos raros (patrón: 2 letras + 10 dígitos + nombre)
    # Ejemplos: ND3335711602, SC3115547608, OD3104793220
    if re.match(r'^[A-Z]{2}\d{10}', desc_str):
        return False
        
    # Filtrar descripciones muy cortas
    if len(desc_lower.strip()) < 3:
        return False
    
    return True

## 3. Aplicar limpieza

In [ ]:
# Aplicar validación
df['is_valid'] = df['vcDescripcion'].apply(is_valid_product)
df_clean = df[df['is_valid'] == True].copy().reset_index(drop=True)

print(f"Original: {len(df)} productos")
print(f"Después de limpieza: {len(df_clean)} productos")
print(f"Removidos: {len(df) - len(df_clean)} productos")

# Ver ejemplos de productos válidos
df_clean[['vcDescripcion']].head(10)

## 4. Guardar resultado

El archivo limpio se guardará como `DetalleFacturas_clean.csv` y puede ser usado por `load_data_to_postgres.py` para cargar productos a la base de datos.

In [ ]:
# Guardar CSV limpio
df_clean.to_csv("../data/DetalleFacturas_clean.csv", index=False, encoding="latin-1")
print("\nArchivo guardado: ../data/DetalleFacturas_clean.csv")
print(f"Total de productos limpios: {len(df_clean)}")

---

## Clasificación de Categorías

**Nota:** La clasificación de categorías se realiza automáticamente por el sistema de IA.

- La API Python (`semantic_search_api.py`) usa **TF-IDF + cosine similarity** para predecir categorías
- Las categorías están definidas en la base de datos PostgreSQL (tabla `categoria`)
- No se requiere procesamiento manual de categorías

### Para usar la API de clasificación:

```bash
# Iniciar la API
python semantic_search_api.py

# Predecir categoría de un producto
curl 'http://localhost:5000/api/predict-category?q=coca%20cola'
```